In [1]:
# Permet de jouer au jeu
import retro
# Permet de ralentir la vitesse 
import time

## Preprocess

In [2]:
from gym import Env
from gym.spaces import MultiBinary, Box
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [3]:
# Environnement Customisé
class StreetFighter(Env) : 
    def __init__(self) :
        super().__init__()
        # A fixer inférieure à 200
        self.taille_reduite = 84
        # Specification de l'espace des actions et de l'espace d'observation
        # low = 0, high = 255 : couleur pixels par défaut
        # shape : shape de la sortie par défaut (hauteur, largeur, Gris)
        self.observation_space = Box(low=0,high=255,shape=(self.taille_reduite,self.taille_reduite,1),dtype=np.uint8)
        # action_space = MultiBinary(12) : 12 touches possibles et combinables pour faire des coups spéciaux
        self.action_space = MultiBinary(12)
        # Lancer une instance du jeu et ne permet que les combinaisons valides de boutons
        self.game = retro.make(game='StreetFighterIISpecialChampionEdition-Genesis', use_restricted_actions = retro.Actions.FILTERED)
    
    def step(self,action):
        # Faire une étape 
        observation, reward, done, info = self.game.step(action)
        observation = self.preprocess(observation)
        
        # Fonction de récompense
        reward = info['score'] - self.score
        self.score = info['score']
        
        return observation, reward, done, info
    
    def render(self,*args,**kwargs):
        self.game.render()
    
    def reset(self):
        # Remet le jeu à zéro
        observation = self.game.reset()
        # Preprocess l'image obtenue
        observation = self.preprocess(observation)
        # Cette variable va permettre de stocker la récompense obtenue pour la partie
        self.score = 0
        return observation
    
    def preprocess(self,observation):
        # Transformation de l'image RGB en nuance de gris => Entraînement plus rapide
        image_gris = cv2.cvtColor(observation,cv2.COLOR_RGB2GRAY)
        # Modifier la taille de l'image => Entraînement plus rapide
        image_retaillee = cv2.resize(image_gris,(self.taille_reduite,self.taille_reduite), interpolation = cv2.INTER_CUBIC)
        # Specificité pour stable_baselines
        image_retaillee_gris_finale = np.reshape(image_retaillee,(self.taille_reduite,self.taille_reduite,1))
        return image_retaillee_gris_finale
    
    def close(self):
        self.game.close()

In [4]:
from stable_baselines3 import PPO
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack
import os

/home/alexandre/INSA/GM5/T2/PBIA/PBIA_Env/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
LOG_DIR = './logs/'
OPT_DIR = './opt/'

## Préparation de l'environnement

In [6]:
# Crée un environnement
env = StreetFighter()
# Permet d'extraire la recompense moyenne et la longueur moyenne d'un episode
env = Monitor(env,LOG_DIR)
#Nécessaire pour Stable Baselines
env = DummyVecEnv([lambda: env])
#Empile 4 images consécutives pour donner la perception de mouvement
env = VecFrameStack(env, 4, channels_order='last')

In [7]:
env.action_space

MultiBinary(12)

In [8]:
env.observation_space

Box([[[0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]
  ...
  [0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]]

 [[0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]
  ...
  [0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]]

 [[0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]
  ...
  [0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]]

 ...

 [[0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]
  ...
  [0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]]

 [[0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]
  ...
  [0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]]

 [[0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]
  ...
  [0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]]], [[[255 255 255 255]
  [255 255 255 255]
  [255 255 255 255]
  ...
  [255 255 255 255]
  [255 255 255 255]
  [255 255 255 255]]

 [[255 255 255 255]
  [255 255 255 255]
  [255 255 255 255]
  ...
  [255 255 255 255]
  [255 255 255 255]
  [255 255 255 255]]

 [[255 255 255 255]
  [255 255 255 255]
  [255 255 255 255]
  ...
  [255 255 255 255]
  [255 255 255 255]
  [255 255 255 255]]

 ...

 [[255 255 255 255]
  [255 255 255 255]
  [255 255 255 255]
  ...
  [255 255 255 255]
  [255 255 255 255]
  [255 255 25

## Test du modèle

In [9]:
SAVE_DIR = './save/'

In [10]:
model = PPO.load(os.path.join(SAVE_DIR,'sf2_best_save_ppo_84x84'))

In [11]:
model.policy

ActorCriticCnnPolicy(
  (features_extractor): NatureCNN(
    (cnn): Sequential(
      (0): Conv2d(4, 32, kernel_size=(8, 8), stride=(4, 4))
      (1): ReLU()
      (2): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2))
      (3): ReLU()
      (4): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
      (5): ReLU()
      (6): Flatten(start_dim=1, end_dim=-1)
    )
    (linear): Sequential(
      (0): Linear(in_features=3136, out_features=512, bias=True)
      (1): ReLU()
    )
  )
  (mlp_extractor): MlpExtractor(
    (shared_net): Sequential()
    (policy_net): Sequential()
    (value_net): Sequential()
  )
  (action_net): Linear(in_features=512, out_features=12, bias=True)
  (value_net): Linear(in_features=512, out_features=1, bias=True)
)

In [ ]:
# Remettre le jeu à zéro
obs = env.reset()
# Partie non terminée
done = False
# Une partie
for game in range(1): 
    while not done: 
        if done: 
            obs = env.reset()
        env.render()
        # Modèle choisit 
        action = model.predict(obs)[0]
        obs, reward, done, info = env.step(action)
        time.sleep(0.01)

In [ ]:
env.close()